In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import datetime
from IPython.display import clear_output
from datetime import timedelta
from scipy.stats import skewnorm
import inspect, re
import h2o
from h2o.automl import H2OAutoML
import xgboost as xgb
               
%load_ext autoreload
%autoreload 2
%matplotlib inline
plt.style.use('ggplot')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
fy_10 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy10-operating-budget.xlsx")
fy_11 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy11-operating-budget.xlsx")
fy_12 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy12-operating-budget.xlsx")
fy_13 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy13-operating-budget.xlsx")
fy_14 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy14-operating-budget.xlsx")
fy_15 = pd.read_excel("budget-vs-actuals-revenue-and-expenses-fy15-operating-budget.xlsx")
fy_16 = pd.read_excel("fy16-budget-vs-actuals-revenue-and-expenses.xls")

#Clean up the formating of FY16
fy_16.columns = fy_16.loc[0].values
fy_16 = fy_16.loc[1:]


In [4]:
fy_list = [fy_10, fy_11, fy_12, fy_13, fy_14, fy_15, fy_16]

for index, year in enumerate(fy_list):
    fy_list[index]['OA_Difference'] = year['Actuals']-year['Original Budget']
    fy_list[index] = year.loc[year['Revenue or Expenditure']=="Expenditures"]
    fy_list[index].dropna(inplace=True)


houston_data = pd.read_csv("Houston Yearly Data - Sheet1 (1).csv")
houston_data_columns = houston_data.iloc[1,1:].values
houston_data_index = houston_data.iloc[2:,0].values
houston_data  = houston_data.iloc[2:,1:]

houston_data.index= houston_data_index
houston_data.columns = houston_data_columns
houston_data.columns = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017',
       '2018', '2019']
houston_data.fillna(0, inplace=True)
houston_data['2019'] = houston_data['2019'].astype(int)

categorical_variables = ['Fund Name', 'Fund Type Descr', 'Business Area Name','Commitment Set Description']


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [5]:
df=pd.concat([x for x in fy_list])

In [6]:
df.loc[df['Actuals']>=0]['Actuals'].sum()/7

4903397655.062859

In [1]:
fy_list[0]['Revenue or Expenditure'].value_counts()

NameError: name 'fy_list' is not defined

In [5]:
def add_houston_data(year_df, houston_data, houston_data_index):
    for col in houston_data_index:
        year_df[col] = 0
    
    budget_year = year_df.iloc[0]['Fiscal Year']
    previous_year =  year_df.iloc[0]['Fiscal Year'] - 1
    
    year_df['Rockets Playoff Rounds'] = float(houston_data[str(previous_year)]['Rockets Playoff Rounds'].replace(',',''))
    year_df['Rockets Total Attendance'] = float(houston_data[str(previous_year)]['Rockets Total Attendance'].replace(',',''))
    year_df['Texans Average Attendance'] = float(houston_data[str(budget_year)]['Texans Average Attendance'].replace(',',''))
    year_df['Texans Playoff Rounds'] = float(houston_data[str(budget_year)]['Texans Playoff Rounds'].replace(',',''))
    year_df['Astros Yearly Attendance'] = float(houston_data[str(previous_year)]['Astros Yearly Attendance'].replace(',',''))
    year_df['Astros Playoff Rounds'] = float(houston_data[str(previous_year)]['Astros Playoff Rounds'].replace(',',''))

    year_df['Yearly Crime Index'] = float(houston_data[str(previous_year)]['Yearly Crime Index'].replace(',',''))
    year_df['Yearly Avg High Temp'] = float(houston_data[str(previous_year)]['Yearly Avg High Temp'].replace(',',''))
    year_df['Yearly Avg Low Temp'] = float(houston_data[str(previous_year)]['Yearly Avg Low Temp'].replace(',',''))
    year_df['Yearly Rain Fall'] = float(houston_data[str(previous_year)]['Yearly Rain Fall'].replace(',',''))
    
    return year_df

In [6]:
for year_index, year in enumerate(fy_list):
    dummies = pd.get_dummies(year[categorical_variables]) # One-hot encoding of categorical variables
    year = pd.concat([dummies, year.drop(categorical_variables, axis=1)], axis=1)
    year.drop('Fund Center Name', axis=1, inplace=True) #This is repeated information
    fy_list[year_index] = add_houston_data(year, houston_data, houston_data_index)
    
shared_columns = list(set(fy_list[0].columns) & set(fy_list[1].columns) & set(fy_list[2].columns) & 
             set(fy_list[3].columns) & set(fy_list[4].columns) & set(fy_list[5].columns) & 
             set(fy_list[6].columns))

for year_index, year in enumerate(fy_list):
    drop_columns = list(set(shared_columns).symmetric_difference(set(fy_list[year_index].columns)))
    fy_list[year_index].drop(drop_columns, axis=1, inplace=True)
    
    
fy_list[6]['Fiscal Year'] = fy_list[6]['Fiscal Year'].astype(int)
fy_list[3]['Original Budget'] = fy_list[3]['Original Budget'].astype(float)
fy_list[4]['Original Budget'] = fy_list[4]['Original Budget'].astype(float)
fy_list[5]['Original Budget'] = fy_list[5]['Original Budget'].astype(float)




In [30]:
# for index, item in enumerate(fy_list):
#     fy_list[index].to_csv('fy_1'+str(index)+'.csv')

In [7]:
# fy_list[0].groupby(['Fiscal Year', 'Fund Id',
#            'Business Area', 'Fund Center Id', 'GL Account', 'GL Description', 'GL Category',
#            'GL Category Description',
#            'Revenue or Expenditure']).groups.keys()

In [8]:
len(shared_columns)

85

In [ ]:
#12310

In [9]:
unique_items_list = []
for index, year in enumerate(fy_list[:6]):
    year_items = []
    for group in fy_list[index].groupby([[''Fund Id',
           'Business Area', 'Fund Center Id', 'GL Account','GL Description', 'GL Category',
           'GL Category Description',
           'Revenue or Expenditure'']]):
        year_items.append(group[0])
    if index==0:
        unique_items_list=list(set(year_items))
    else:
        unique_items_list =list(set(year_items) & set(unique_items_list))
        

len(unique_items_list)
    

12310

In [10]:
unique_items_df = pd.DataFrame()
unique_items_df['key']= unique_items_list
unique_items_df.to_csv('unique_items.csv')

In [132]:
unique_items_list[0] in unique_items_list[:2]

True

In [134]:
unique_df = pd.DataFrame(columns=fy_list[0].columns)
for index, year in enumerate(fy_list[:6]):
    for group in fy_list[index].groupby(['Fund Id',
           'Business Area', 'Fund Center Id', 'GL Account','GL Description', 'GL Category',
           'GL Category Description',
           'Revenue or Expenditure']):
        if group[0] in unique_items_list:
            unique_df = pd.concat([unique_df, group[1]])
            
            

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [148]:
6*12310

73860

In [150]:
unique_df.head()

,Actuals,Astros Playoff Rounds,Astros Yearly Attendance,Business Area,Business Area Name_Admn. & Regulatory Affairs,Business Area Name_Business Opportunity,Business Area Name_City Controller's Office,Business Area Name_City Council,Business Area Name_City Secretary,Business Area Name_Citywide General Government,Business Area Name_Convention and Entertainment,Business Area Name_Finance,Business Area Name_General Debt Services,Business Area Name_General Services Department,Business Area Name_Health & Human Services,Business Area Name_Housing and Community Devp.,Business Area Name_Houston Airport System (HAS),Business Area Name_Houston Emergency Center (HEC),Business Area Name_Houston Fire Department (HFD),Business Area Name_Human Resources Dept.,Business Area Name_Legal Department,Business Area Name_Library,Business Area Name_Mayor's Office,Business Area Name_Municipal Courts Department,Business Area Name_Parks & Recreation,Business Area Name_Planning & Development,Business Area Name_Public Works & Engineering-PWE,Business Area Name_Solid Waste Management,Commitment Set Description_50 - Total Expenditures,Current Budget,Fiscal Year,Fund Center Id,Fund Id,Fund Name_Asset Forfeiture Fund - Justice,Fund Name_Asset Forfeiture Fund - State,Fund Name_Asset Forfeiture Fund - Treasury,Fund Name_Auto Dealers Fund,Fund Name_Building Inspection Fund,Fund Name_C&E-Civic Center Facility Revenue Fund,Fund Name_Cable Television Fund,Fund Name_Child Safety Fund,Fund Name_Digital Houston Special Revenue,Fund Name_Expansion of Recycling Program,Fund Name_Fleet Management Fund,Fund Name_GSD - In-House Renovation Revolving Fund,Fund Name_General Fund,Fund Name_Greater Houston Trans & Emer Center,Fund Name_HAS-Revenue Fund,Fund Name_Health Benefits Fund,Fund Name_Historic Preservation Fund,Fund Name_Houston Emergeny Center,Fund Name_Juvenile Case Manager Fee,Fund Name_Long-Term Disablility Plan Fund,Fund Name_Municipal Court Bldg Security Fund,Fund Name_Municipal Court Technology Fund,Fund Name_PWE-Combined Utility System Gen Pur Fund,Fund Name_PWE-Combined Utility System Oper Fund,Fund Name_PWE-W & S System Operating Fund,Fund Name_Park Special Revenue Fund,Fund Name_Parking Mangement Operating Fund,Fund Name_Police Special Services,Fund Name_Project Cost Recovery,Fund Name_Property & Casualty Fund,Fund Name_Stormwater Fund,Fund Name_Supplemental Environmental Proj,Fund Name_Workers Compensation Admin Fund,Fund Type Descr_Enterprise Funds,Fund Type Descr_General Funds,Fund Type Descr_Internal Services Funds,Fund Type Descr_Special Revenue Funds,GL Account,GL Category,GL Category Description,GL Description,OA_Difference,Original Budget,Revenue or Expenditure,Rockets Playoff Rounds,Rockets Total Attendance,Texans Average Attendance,Texans Playoff Rounds,Yearly Avg High Temp,Yearly Avg Low Temp,Yearly Crime Index,Yearly Rain Fall,key
1,960305.36,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1831618.0,2010,1000010001,1000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500010,500,500 - Personnel Services,500010 - Salary Base Pay - Civilian,-37155.64,997461.0,Expenditures,0.0,699280.0,71080.0,0.0,80.0,62.0,596.6,47.01,none
2,1258680.84,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1320119.0,2010,1000010001,1000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500020,500,500 - Personnel Services,500020 - Salary Base Pay - Classified,-61438.16,1320119.0,Expenditures,0.0,699280.0,71080.0,0.0,80.0,62.0,596.6,47.01,none
4,70887.50,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,75000.0,2010,1000010001,1000,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,500050,500,500 - Personnel Services,500050 - Salary-Education/Incentive Pay - Classif,-4112.50,75000.0,Expenditures,0.0,699280.0,71080.0,0.0,80.0,62.0,596.6,47.01,none
5,12191.97,0.0,2521076.0,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,10800.0,2010,1000010

In [136]:
len(unique_df)

73860

In [137]:
unique_df.to_csv("unique_df.csv")

In [140]:
year_list = [2010, 2011, 2012, 2013, 2014, 2015, 2016]

In [141]:
for index, year in enumerate(fy_list[:6]):
    fy_list[index] = df.loc[df['Fiscal Year']==year_list[index]]

In [142]:
for index, item in enumerate(fy_list[:6]):
    fy_list[index].to_csv('fy_1'+str(index)+'unique'+'.csv')

In [143]:
actuals_list = []
for index, year in enumerate(fy_list[:6]):
    actuals_list.append(year['Actuals'])

In [144]:
for index, year in enumerate(fy_list[1:5]):
    
    #if in 2012 lookback two years
    if index==1:
        fy_list[index]['3Y_Past_Actuals']="na"
        fy_list[index]['2Y_Past_Actuals']=actuals_list[index-2]
        fy_list[index]['1Y_Past_Actuals']=actuals_list[index-1]
        hold_df = fy_list[index][["2Y_Past_Actuals","1Y_Past_Actuals"]]
        fy_list[index]['2Y_1Y_Mean'] = hold_df.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Mean'] = "na"
        
    #otherwise lookback three years
    else:
        fy_list[index]['3Y_Past_Actuals']=actuals_list[index-3]
        fy_list[index]['2Y_Past_Actuals']=actuals_list[index-2]
        fy_list[index]['1Y_Past_Actuals']=actuals_list[index-1]
        hold_df_2 = fy_list[index][["2Y_Past_Actuals","1Y_Past_Actuals"]]
        hold_df_3 = fy_list[index][["3Y_Past_Actuals","2Y_Past_Actuals","1Y_Past_Actuals"]]
        fy_list[index]['2Y_1Y_Mean'] = hold_df_2.mean(axis=1)
        fy_list[index]['3Y_2Y_1Y_Mean'] = hold_df_2.mean(axis=1)
    

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-

In [145]:
for index, item in enumerate(fy_list[:6]):
    fy_list[index].to_csv('fy_1'+str(index)+'unique_PYA'+'.csv')



In [ ]:
#6:09pm 9-3-19